In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix
from skimage.measure import shannon_entropy
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio

In [3]:
path = "lena.bmp"

def read(file):
        file = cv2.imread(file)
        ycrcb = cv2.cvtColor(file, cv2.COLOR_BGR2YCR_CB)
        Y, CR, CB = cv2.split(ycrcb) 
        return Y, CR, CB, file

def cut_img(img, size):  # cut_img(img, n等分)
    width, height = np.shape(img)
    item_width = int(width / size)
    img_list = []
    for i in range(0,size):
        for j in range(0,size):
            new_img = img[i*item_width:(i+1)*item_width, j*item_width:(j+1)*item_width]
            img_list.append(new_img)
    return img_list

def GLCM(grayImg, distances, angles):
	glcm = greycomatrix(grayImg, distances=distances, angles=angles, symmetric=True, normed=True)
	return glcm

def entropy(cut_img):
        entropy_list = []
        entropy_sum = 0
        for i in range(0, np.shape(cut_img)[0]):
                glcm = GLCM(cut_img[i], distances=[1], angles=[0])
                e = shannon_entropy(glcm)
                entropy_list.append(e)
                entropy_sum += e
        return entropy_list, entropy_sum


# read file
Y, CR, CB, file = read(path)
print("Y_shape = ", np.shape(Y))

# 原圖大小
original_size = np.shape(Y)[0]
print("original_size = ", original_size)
# 裁切大小
cut_size = 8
print("cut_size = ", cut_size)

# 原圖1024x1024 切成16384個8x8 每邊128
cut_imgs = cut_img(Y, int(original_size / cut_size))
entropy_list, entropy_sum = entropy(cut_imgs)
entropy_average = entropy_sum / np.shape(cut_imgs)[0]

print("cut_imgs =", np.shape(cut_imgs))
print("entropy_list =", np.shape(entropy_list))


# 計算 entropy > average 有幾個，回傳list裡面是0或1
block_list = []
count = 0
for i in entropy_list:
        if i > entropy_average:
                block_list.append(1)
                count += 1
        else:
                block_list.append(0)
# print(block_list) # 16384個值
print("block_list =", np.shape(block_list))
print("block_list_one=", count) # 8532個1



Y_shape =  (1024, 1024)
original_size =  1024
cut_size =  8
cut_imgs = (16384, 8, 8)
entropy_list = (16384,)
block_list = (16384,)
block_list_one= 8532


In [4]:
# 裁切大小
sub_cut_size = 16
number_of_cut = int((original_size / sub_cut_size)**2)
equal_parts = int(original_size / sub_cut_size)

grid_imgs = cut_img(Y, int(original_size / sub_cut_size)) # 16x16 4096張 1024 => 64等分
print("grid_img =", np.shape(grid_imgs))
# for i in range(0,np.shape(grid_imgs)[0]):
#     cv2.imwrite('C:\Python\HW4\dip_new\grid_img\%d.bmp' %i, grid_imgs[i])

block_size = int(original_size / cut_size)
block_list = np.reshape(block_list, (block_size, block_size)) # 128x128 8x8一個值
print("block_list =", np.shape(block_list))

# 數範圍內有多少個1
def count_one(block_list, a, b, c, d):
    count = 0
    for i in range(a,b):
        for j in range(c,d):
            if block_list[i][j]==1:
                count +=1
    return count


# 分成nxn一組
n = 2

# 把128x128分成每2x2一組
list_1 = [i for i in range(0,block_size,n)]
list_2 = [i for i in range(0,block_size,n)]

number_sum = 0
number_list = []
for i in list_1:
    for j in list_2:
        number = count_one(block_list, i, i+n, j, j+n)
        number_sum += number
        number_list.append(number)
        # print((i, i+4, j, j+4), "count=", number)

# print(number_list)
print("number_list =", np.shape(number_list))
number_average = number_sum / number_of_cut
print("number_average =", number_average)


# 把大於平均的標上1，小於平均標上0
number_index = []
for i in number_list:
    if i>number_average:
        number_index.append(1)
    else:
        number_index.append(0)
# print(number_index)
number_index = np.reshape(number_index, (equal_parts, equal_parts))
print("number_index =", np.shape(number_index))


# 邊緣拿掉
for i in range(0,equal_parts):
    for j in range(0,equal_parts):
        if i==0 or j==0 or i==equal_parts-1 or j==equal_parts-1:
            number_index[i][j]=0
# print(number_index)
print(np.shape(number_index))
number_index = np.reshape(number_index, (number_of_cut))
print(np.shape(number_index))


# 值等於1的index
out_index = np.where(number_index==1)


# 選出的texture region
count = 0
out_list = []
for i in out_index[0]:
    count += 1
    out = grid_imgs[i]
    out_list.append(out)
    #cv2.imwrite('%d.bmp' %i, out)
print("out_list =", np.shape(out_list))

# 加上邊框的texture region
border_list = []
for i in out_index[0]:
    new = grid_imgs[i][1:sub_cut_size-1, 1:sub_cut_size-1]
    border = cv2.copyMakeBorder(new, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
    #cv2.imwrite('%d.bmp' %i, border)
    border_list.append(border)

print("border_list=", np.shape(border_list))



grid_img = (4096, 16, 16)
block_list = (128, 128)
number_list = (4096,)
number_average = 2.0830078125
number_index = (64, 64)
(64, 64)
(4096,)
out_list = (1783, 16, 16)
border_list= (1783, 16, 16)


In [5]:
# 把加上邊框的texture region加上原圖輸出
grid_imgs_border = grid_imgs
for i in range(0, number_of_cut):
        for j in out_index[0]:
                if i == j:
                        new = grid_imgs_border[j][1:sub_cut_size-1, 1:sub_cut_size-1]
                        border = cv2.copyMakeBorder(new, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
                        grid_imgs_border[j] = border
print(np.shape(grid_imgs_border))

new = np.ones((original_size, original_size))
list_1 = [i for i in range(0, original_size, sub_cut_size)]
list_2 = [i for i in range(0, original_size, sub_cut_size)]
m=0
for i in list_1:
        for j in list_2:
                new[i:i+sub_cut_size, j:j+sub_cut_size] = grid_imgs_border[m]
                m += 1

print("new =", np.shape(new))
cv2.imwrite("grided.bmp", new)

(4096, 16, 16)
new = (1024, 1024)


True

In [6]:
# 隨機挑選block中m個圖
m = 841  # watermark 29x29
choice = np.random.choice(out_index[0], m, replace=False)
print(choice)

grid_imgs = cut_img(Y, int(original_size / sub_cut_size)) 

count = 0
for i in choice:
    count += 1
    save = grid_imgs[i]
    #cv2.imwrite("C:\Python\HW4\dip_new\choice_img\choice%d.bmp"%i, save)
print(count)


[3694 3528 3033 1502 2900  477 2636 2646  119 3529 3779 2540 3916 1880
 3094 3222 3985 3119  271  233 2084  574  764 1111 1020 1586 3915  151
 3472 1437 1481 1780 2191 1620 1823 3895 2154 1106 2265  583 2372 2894
 1306 1629 3486  613 1040 2538 2123 1143 2955 1392 3343 2204 3184 3666
 1489 1566 2605 4006 3792 2892  311 3203 3224 2311  790 3416 1375 1706
 3147  135 2353 3844  302 2698 2861 2882 1043 3611 2030 2051 1505  795
 2899 3117  949 1031 2008 1468 3353 3422 2125 1808 3344  730  797  938
 1371 1779 2649  783 3595 3374 1304 2381 2193 2627 2522  721 2710  942
 2983 3796 2137 2327 2729 3243 3917 2582 2413 1618 1525 2835 3082 3358
  952 2073 3851 1364  702 3790 3566 1652 3513 2316 2696 3339 3414 1495
 3803 1795 3267 2507 2284  990 3210 2202 1775 3254   78 1617 3027 1632
 2077  883 1517 2070  451 2984 2013  440 3858 3921 1991 2820 1684  554
   66 1313 1587  600 1622 1045 2383 2417 1008 2576  527 3850 3631 3026
 3524 1311 3400 2714  231 3204 1347 3705 2133 1114  941 1760 2132 1667
  988 

In [7]:
#DCT

def mask(img, size):
    h, w = np.shape(img)
    mask = np.zeros((h,w), np.uint8)
    mask[0:size, 0:size] = 1   # 保留左上角 nxn
    img = img * mask 
    spectrum = np.abs(img)
    return  img, spectrum


def matrix_A(masked):
    A = np.zeros((4,4))
    for i in range(0,4):
        for j in range(0,4):
            A[i][j] = masked[i][j]
    return A

print("m = ", m)

total_dct_img = np.zeros((m,16,16))
total_A = np.zeros((m,4,4))
count=0
for i in choice:

    dct_img = cv2.dct(np.float32(grid_imgs[i]))
    dct_magnitude_spectrum = np.abs(dct_img)
    masked, masked_magnitude_spectrum = mask(dct_img, 4)
    A = matrix_A(masked)

    # 存下所有的dct_img, A 各m個
    total_dct_img[count,:,:] = dct_img.copy()
    total_A[count,:,:] = A.copy()
    count+=1


m =  841


In [8]:
# watermark
watermark_path = "watermark.png"
w_file = cv2.imread(watermark_path)
w_ycrcb = cv2.cvtColor(w_file, cv2.COLOR_BGR2YCR_CB)
Y_w, CR_w, CB_w = cv2.split(w_ycrcb) 


# resize
size = 29
resize_watermark = cv2.resize(Y_w, (size,size))
ret, out = cv2.threshold(resize_watermark, 127, 255, cv2.THRESH_BINARY)
# cv2.imwrite("watermark_resize.png", out)


watermark_list = []
for i in range(0, size):
    for j in range(0, size):
        if 0 == out[i][j]:
            watermark_list.append(0)
        else:
            watermark_list.append(255)
print(np.shape(watermark_list))

# print('---Y_w----')
# print(Y_w)
# print('--CR---')
# print(CR_w)
# print('--CB---')
# print(CB_w)


(841,)


In [9]:
# QR decomposition function
def QR_decomposition(A):
    N = len(A)
    q = np.zeros((4,4))
    r = np.zeros((4,4))
    M = np.matrix.dot(A.transpose(),A)

    r11_squre = M[0][0]
    if r11_squre <= 0:
        r[0][0] = 1
    else:
        r[0][0] = np.sqrt(r11_squre)

    r[0][1] = M[0][1] / r[0][0]
    r[0][2] = M[0][2] / r[0][0]
    r[0][3] = M[0][3] / r[0][0]

    # 避免根號內負數或為0，若r22=0會造成r23,r23變成無窮大
    # r22<0同樣也會造成無窮大
    r22_square = M[1][1] - r[0][1]**2
    if r22_square <= 0:
        r[1][1] = 1
    else:
        r[1][1] = np.sqrt(r22_square)

    r[1][2] = (M[1][2] - r[0][1]*r[0][2]) / r[1][1]
    r[1][3] = (M[1][3] - r[0][1]*r[0][3]) / r[1][1]

    
    r33_square = M[2][2] - r[0][2]**2 - r[1][2]**2
    if r33_square <= 0:
        r[2][2] = 1
    else:
        r[2][2] = np.sqrt(r33_square)

    r[2][3] = (M[2][3] - r[0][2]*r[0][3] - r[1][2]*r[1][3]) / r[2][2]

    r44_square = M[3][3] - r[0][3]**2 - r[1][3]**2 - r[2][3]**2
    if r44_square <= 0:
        r[3][3] = 1
    else:
        r[3][3] = np.sqrt(r44_square)
    
    # calculate Q use Gram-Schmidt algorithm
    q[:,0] = A[:,0] / r[0,0]
    q[:,1] = (A[:,1] - r[0,1]*q[:,0]) / r[1,1]
    q[:,2] = (A[:,2] - r[0,2]*q[:,0] - r[1,2]*q[:,1]) / r[2,2]
    q[:,3] = (A[:,3] - r[0,3]*q[:,0] - r[1,3]*q[:,1] - r[2,3]*q[:,2]) / r[3,3]


    return q,r

In [10]:
# QR分解， embedding watermark
q = 350

def replace_r11_0(r11, q):
    z = r11 % q
    if z <= 3*(q / 4):
        new_r11 = r11 + (q/4) - z
    else:
        new_r11 = r11 + 5*(q/4) - z

    return new_r11

def replace_r11_255(r11, q):
    z = r11 % q
    if z <= (q / 4):
        new_r11 = r11 - (q/4) - z
    else:
        new_r11 = r11 + 3*(q/4) - z

    return new_r11

def insert_dct_A(img,A):
    for i in range(0,4):
        for j in range(0,4):
            img[i][j] = A[i][j]
    
    return img
        

total_A_prime = total_A.copy()
total_dct_img_prime = total_dct_img.copy()
for i in range(0, m):
    #Q, R = np.linalg.qr(total_A[i])
    Q, R = QR_decomposition(total_A[i])
    R_prime = R.copy()
    if watermark_list[i] == 0:
        #R[0][0] = R[0][0]
        R_prime[0][0] = replace_r11_0(R[0][0], q)
    else:
        #R[0][0] = R[0][0]
        R_prime[0][0] = replace_r11_255(R[0][0], q)
    total_A_prime[i] = np.dot(Q,R_prime)
    #print("total_A_prime=",total_A_prime[i])
    total_dct_img_prime[i] = insert_dct_A(total_dct_img[i],total_A_prime[i])

print("done")
    
    

done


In [11]:
# inverse dct
m = 841
total_idct_img = np.zeros((m, 16, 16))
for i in range(0, m):
        total_idct_img[i] = cv2.idct(total_dct_img_prime[i])


# add back to grid_imgs
grid_imgs = cut_img(Y, int(original_size / sub_cut_size)) # 16x16 4096張 1024 => 64等分
count = 0
embedded_grid_imgs = grid_imgs.copy()
for i in choice:
    embedded_grid_imgs[i] = total_idct_img[count]
    count += 1
print(np.shape(grid_imgs))
print(np.shape(embedded_grid_imgs))



embedded_img = np.ones((original_size, original_size))
list_1 = [i for i in range(0, original_size, sub_cut_size)]
list_2 = [i for i in range(0, original_size, sub_cut_size)]
m=0
for i in list_1:
        for j in list_2:
                embedded_img[i:i+sub_cut_size, j:j+sub_cut_size] = embedded_grid_imgs[m].copy()
                m += 1

print("embedded_img =", np.shape(embedded_img))
cv2.imwrite("embedded.bmp", embedded_img)

(4096, 16, 16)
(4096, 16, 16)
embedded_img = (1024, 1024)


True

In [12]:
# output.bmp
def write(Y,CR,CB):
        output_ycrcb = cv2.merge([Y,CR,CB])
        output_img = cv2.cvtColor(output_ycrcb,cv2.COLOR_YCR_CB2BGR)
        return output_img

embedded_img = np.round(embedded_img)
Y_prime = embedded_img.astype(np.uint8)
# Y_prime = cv2.normalize(embedded_img, None, 16, 235, cv2.NORM_MINMAX, cv2.CV_8U)
output = write(Y_prime,CR,CB)
print("output =", np.shape(output))
cv2.imwrite("output.bmp", output)

output = (1024, 1024, 3)


True

In [13]:
# PSNR

# import lena.bmp
path = "lena.bmp"
file = cv2.imread(path)

# import output.bmp
path2 = "output.bmp"
file2 = cv2.imread(path2)

# PSNR
PSNR = peak_signal_noise_ratio(file, file2) # file is lena.bmp
print("PSNR=",PSNR)

# SSIM
SSIM = structural_similarity(file, file2, multichannel=True)
print("SSIM",SSIM)

# plot figure
# plt.figure()
# plt.title("lena")
# plt.imshow(cv2.cvtColor(file,cv2.COLOR_BGR2RGB))
# plt.figure()
# plt.title("output")
# plt.imshow(cv2.cvtColor(file2,cv2.COLOR_BGR2RGB))

PSNR= 38.842875674030005
SSIM 0.9853245699976977


In [18]:
# extraction
# idct_img block do dct and get matrix A

def matrix_A2(img):
    A = np.zeros((4,4))
    for i in range(0,4):
        for j in range(0,4):
            A[i][j] = img[i][j]
    return A

def R_to_W(r11,q):
    z = r11 % q
    if z <= q/2:
        w = 0
    else:
        w = 255
    return w

m = 841

e_path = "output.bmp"
# e_path = "gaussian_noise_out.bmp"
# e_path = "gaussian_blur_out.bmp"
# e_path = "compress_out.bmp"
e_file = cv2.imread(e_path)


if e_file.shape[0] != original_size:
    e_file = cv2.resize(e_file, (1024,1024))


### test resize
# plt.figure(figsize=(15,15)) 
# test_image_rgb = cv2.cvtColor(e_file, cv2.COLOR_BGR2RGB)
# plt.imshow(test_image_rgb)
# plt.show()
# ###


e_ycrcb = cv2.cvtColor(e_file, cv2.COLOR_BGR2YCR_CB)
Y_e, CR_e, CB_e = cv2.split(e_ycrcb)
print("Y_e=",np.shape(Y_e))

original_size = np.shape(Y_e)[0]
#sub_cut_size = 16
grid_imgs_e = cut_img(Y_e, int(original_size / sub_cut_size))
print("grid_imgs_e=",np.shape(grid_imgs_e))
#print(choice)
print("choice.size=",choice.size)
extract_img = np.zeros((m,16,16))
extract_dct_img = np.zeros((m,16,16))

count=0
for i in choice:
    extract_img[count] = grid_imgs_e[i].copy()
    count += 1
print("extract_img= ", np.shape(extract_img))
print("m=",m)
# print(extract_img[0,0:4,0:4])
# print(grid_imgs_e[choice[0]][0:4,0:4])
# print("----")
# print(total_idct_img[0,0:4,0:4])
# print(grid_imgs[choice[0]][0:4,0:4])
# print("----")

# extract_dct_img = total_idct_img.copy()
# print("total_idct_img_shape=",np.shape(total_idct_img))
#print("extract_dct_img_shape=",np.shape(extract_dct_img))
A_e = np.zeros((m,4,4))
count=0
for i in range(0,m):
    #extract_dct_img[i] = cv2.dct(np.float32(total_idct_img[i]))
    extract_dct_img[i] = cv2.dct(np.float32(extract_img[i]))
    A_e[count] = matrix_A2(extract_dct_img[i])
    count += 1

#q = 40
watermark_e_list = []
for i in range(0,m):
    #Q, R = np.linalg.qr(A_e[i])
    Q, R = QR_decomposition(A_e[i])
    r11 = R[0][0]
    watermark_e_list.append(R_to_W(r11,q))
#print(watermark_list)
#print("vs")
#print(watermark_e_list)
if(watermark_list == watermark_e_list):
    print("Lists are equal")
else:
    print("Lists are not equal")
print(np.shape(watermark_list))
print("watermark__e_list_shape=",np.shape(watermark_e_list))
watermark_e = np.reshape(watermark_e_list,(29,29,1))
print("watermark_e_shape=",np.shape(watermark_e))


## resize watermark_e (extraction) and save as "watermark_e.bmp"
watermark_e = watermark_e.astype(np.uint8)
resize_watermark_e = cv2.resize(watermark_e, (290,290),interpolation=cv2.INTER_NEAREST)
ret, thresh = cv2.threshold(resize_watermark_e, 127, 255, cv2.THRESH_BINARY)
cv2.imwrite("watermark_e.png", thresh)  
print("thresh=",np.shape(thresh)) 
    


Y_e= (1024, 1024)
grid_imgs_e= (4096, 16, 16)
choice.size= 841
extract_img=  (841, 16, 16)
m= 841
Lists are not equal
(841,)
watermark__e_list_shape= (841,)
watermark_e_shape= (29, 29, 1)
thresh= (290, 290)


In [15]:
# BER

watermark_path = "watermark.png"
w_file = cv2.imread(watermark_path)
w_ycrcb = cv2.cvtColor(w_file, cv2.COLOR_BGR2YCR_CB)
Y_w, CR_w, CB_w = cv2.split(w_ycrcb) 


watermark_e_path = "watermark_e.png"
w_e_file = cv2.imread(watermark_e_path)
w_e_ycrcb = cv2.cvtColor(w_e_file, cv2.COLOR_BGR2YCR_CB)
Y_w_e, CR_w_e, CB_w_e = cv2.split(w_e_ycrcb) 



count = 0
for i in range(0, np.shape(Y_w)[0]):
    for j in range(0, np.shape(Y_w)[0]):
        if Y_w[i][j] != Y_w_e[i][j]:
            count += 1

print(count)
BER = (count / np.shape(Y_w)[0]**2) * 100
print(BER,"%")


0
0.0 %


In [139]:
# 測試 r11,這整段可以註解掉
# extract_dct_img_test1 = np.zeros((1024,16,16))
# extract_dct_img_test2 = np.zeros((1024,16,16))
# extract_dct_img_test1[0] = cv2.dct(np.float32(total_idct_img[0]))
# extract_dct_img_test2[0] = cv2.dct(np.float32(extract_img[0]))
# print("---total_idct_img---")
# print(total_idct_img[0,0:4,0:4])
# print("---extract_img---")
# print(extract_img[0,0:4,0:4])
# print("---extract_dct_img_test1---")
# print(extract_dct_img_test1[0,0:4,0:4])
# print("---extract_dct_img_test2---")
# print(extract_dct_img_test2[0,0:4,0:4])

# A_e_test1 = matrix_A2(extract_dct_img_test1[0])
# A_e_test2 = matrix_A2(extract_dct_img_test2[0])

# #Q_test1, R_test1 = np.linalg.qr(A_e_test1)
# Q_test1, R_test1 = QR_decomposition(A_e_test1)
# r11_test1 = R_test1[0][0]

# #Q_test2, R_test2 = np.linalg.qr(A_e_test2)
# Q_test2, R_test2 = QR_decomposition(A_e_test2)
# r11_test2 = R_test2[0][0]

# print("---r11_test1---")
# print(r11_test1)
# print("---r11_test2---")
# print(r11_test2)


In [140]:
# # output watermark_extract.bmp
# def write(Y,CR,CB):
#         output_ycrcb = cv2.merge([Y,CR,CB])
#         output_img = cv2.cvtColor(output_ycrcb,cv2.COLOR_YCR_CB2BGR)
#         return output_img



# # resize
# '''
# 有問題!!我們取的watermark是32x32,若將其放大到290會糊糊的我的手機沒辦法掃出QRcode(但32x32可以掃得出來)
# 可是仔細看取出的watermark_extract.bmp和watermark.png比較會發現兩者不太相同
# 我目測主要是QRcode外圍多了一圈,就是原本QRcode的方框向外延伸了一格的感覺,左上角的正方形左方上方都厚了一點
# '''
# w_size = 32
# resize_CR = np.zeros((w_size,w_size)) +128 #縮小成32x32
# resize_CB = np.zeros((w_size,w_size)) + 128
# #resize_CB = cv2.resize(CB_w, (32,32)) #縮小成32x32
# resize_CR = resize_CR.astype(np.uint8)
# resize_CB = resize_CB.astype(np.uint8)
# watermark_e = watermark_e.astype(np.uint8)
# #resize_watermark_e = cv2.resize(watermark_e, (290,290),interpolation=cv2.INTER_LINEAR) #放大成290x290
# resize_watermark_e = cv2.resize(watermark_e, (w_size,w_size),interpolation=cv2.INTER_LINEAR) #縮小成32x32

# Y_e_prime = resize_watermark_e.astype(np.uint8)
# #Y_prime = cv2.normalize(Y_e_prime, None, 16, 235, cv2.NORM_MINMAX, cv2.CV_8U)
# #watermark_extract = write(Y_e_prime,CR_w,CB_w) #放大成290x290 YCr_Cb to BGR
# watermark_extract = write(Y_e_prime,resize_CR,resize_CB)
# print("output =", np.shape(watermark_extract))
# cv2.imwrite("watermark_extract.bmp", watermark_extract)

In [141]:
# # 手動選擇一些texture region
# chosen = [(4,6),(5,6),(6,6),
# (4,7),(5,7),(6,7),
# (2,8),(3,8),(4,8),(5,8),(6,8),
# (2,9),(3,9),(4,9),(5,9),(6,9),
# (2,10),(3,10),(4,10),(5,10),(6,10),
# (2,11),(3,11),(4,11),(5,11),(6,11),
# (2,12),(3,12),(4,12),(5,12),(6,12),
# (2,13),(3,13),(4,13),(5,13),(6,13),
# (2,14),(3,14),(4,14),(5,14),(6,14),
# (11,6),(11,7),(11,8),(11,9),(11,10),(11,11),(11,12),(11,13)]


# chosen_matrix = np.zeros((16,16)) 

# # 16x16的矩陣中，把選擇的texture region標上1
# for i in chosen:
#     a = i[0]
#     b = i[1]
#     chosen_matrix[b][a] = 1

# # 輸出選擇的texture region座標
# count = -1
# count_index = []
# for i in range(0,16):
#     for j in range(0,16):
#         count += 1
#         if 1 == chosen_matrix[i][j]:
#             count_index.append(count)
# print(count_index)




In [142]:
# grid_imgs_new = cut_img(Y, 16)
# print(np.shape(grid_imgs_new))

# a = np.zeros((256, 64, 64))
# b = np.zeros((256, 64, 64))
# out = np.stack((grid_imgs_new, a, b), axis=3)
# print(np.shape(out))

# #SIFT
# sift = cv2.SIFT_create()

# for i in range(0,256):
#         for j in count_index:
#                 if i == j:
#                         kp = sift.detect(grid_imgs_new[j], None)
#                         img = cv2.drawKeypoints(grid_imgs_new[j], kp, file, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#                         out[j] = img

# print(np.shape(out))

# new = np.ones((1024, 1024, 3))
# list_1 = [i for i in range(0,1024,64)]
# list_2 = [i for i in range(0,1024,64)]
# m=0
# for i in list_1:
#         for j in list_2:
#                 new[i:i+64, j:j+64] = out[m]
#                 m += 1

# print("new =", np.shape(new))
# # cv2.imwrite("C:\Python\HW4\out_new.bmp", new)

In [143]:
# SIFT

# count = 0
# for i in count_index:
#         count += 1
#         kp = sift.detect(grid_imgs_new[i], None)
#         kp, des = sift.detectAndCompute(grid_imgs_new[i],None)
# print(count)

# kp = sift.detect(grid_imgs_new[100], None)

# kp, des = sift.detectAndCompute(grid_imgs_new[100], None)
# print(des.shape)


# def filter(i):
#     n = np.where(des[i]<100)
#     for j in n[0]:
#         des[i][j] = 0
#     return des[i]

# # a = np.zeros((25,128))
# for i in range(0,25):
#     a[i] = filter(i)

# print(np.shape(grid_imgs_new[0]))
# img = cv2.drawKeypoints(a, kp, file, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# cv2.imwrite("C:\Python\HW4\_aaaaa.bmp", img)
# for i in range(0,25):
#     print(des[i])
#     print(np.sqrt((des[i]*des[i]).sum()))

# sum = 0
# for i in des[0]:
#     sum += i**2
# print(sum)
# print(np.sqrt(sum))


# kp = sift.detect(grid_imgs_new[100], None)
# img = cv2.drawKeypoints(grid_imgs_new[100], kp, file, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# cv2.imwrite("C:\Python\HW4\_100.bmp", img)